# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Изучение файла

In [1]:
# импортируем необходимые библиотеки
import pandas as pd
import numpy as np 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.dummy import DummyClassifier

In [2]:
# откроем и изучим файл
df = pd.read_csv('/datasets/users_behavior.csv')

print(df.shape)
print(df.head(5))

(3214, 5)
   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0


Данные загружены и изучены. В нашем распоряжении обучающий набор данных и целевой признак, который нужно предсказать по остальным признакам, 
— `is_ultra` подходящий тариф. Такие задачи относятся к классу «обучение с учителем» (от англ. supervised learning). «Учитель» 
ставит вопросы (признаки) и указывает ответы (целевой признак). Если целевой признак категориальный, то решается задача классификации.

## Разбивка данных на выборки

Данные нужно разбить на три части: обучающую, валидационную и тестовую. 
Размеры тестового и валидационного наборов обычно равны. Исходные данные разбивают в соотношении 3:1:1 (60,20,20).
Тестовая выборка нужна, чтобы правильно оценить готовую модель.

In [3]:
# разделим исходные данные на обучающую, валидационную и тестовую выборки

df_train, df_valid = train_test_split(df, test_size=0.4, random_state=12345)
df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=12345)

print("Размер обучающей выборки:", df_train.shape)
print("Размер валидационной выборки:", df_valid.shape)
print("Размер тестовой выборки:", df_test.shape)

Размер обучающей выборки: (1928, 5)
Размер валидационной выборки: (643, 5)
Размер тестовой выборки: (643, 5)


Данные разбиты на три выборки.

## Исследование модели

Объявим четыре переменные и запишим в них:
признаки: features_train, features_valid;
целевой признак: target_train, target_valid.

In [4]:
features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop('is_ultra', axis=1)
target_valid = df_valid['is_ultra']

print(features_train.shape)
print(features_valid.shape)

(1928, 4)
(643, 4)


In [5]:
# модель решающее дерево

best_model = None
best_result = 0
for depth in range(1, 6):
	model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # обучите модель с заданной глубиной дерева
	model.fit(features_train, target_train) # обучим модель
	result = model.score(features_valid, target_valid) # посчитаем качество модели
	if result > best_result:
		best_model = model
		best_result = result
        
print("Accuracy лучшей модели решающее дерево на валидационной выборке:", best_result)

Accuracy лучшей модели решающее дерево на валидационной выборке: 0.7853810264385692


In [6]:
# модель случайный лес

best_model = None
best_result = 0
for est in range(10, 51, 10):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) # обучим модель с заданным количеством деревьев
    model.fit(features_train, target_train) # обучим модель на тренировочной выборке
    result = model.score(features_valid, target_valid) # посчитаем качество модели на валидационной выборке
    if result > best_result:
        best_model = model # сохраним наилучшую модель
        best_result = result #  сохраним наилучшее значение метрики accuracy на валидационных данных

print("Accuracy наилучшей модели случайный лес на валидационной выборке:", best_result)

Accuracy наилучшей модели случайный лес на валидационной выборке: 0.7916018662519441


In [7]:
# логистическая регрессия

model = LogisticRegression(random_state=12345) # инициализируйте модель логистической регрессии с параметром random_state=12345
model.fit(features_train, target_train) # обучите модель на тренировочной выборке
result = model.score(features_valid, target_valid) # получите метрику качества модели на валидационной выборке

print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7107309486780715


In [8]:
# улучшим модель случайного леса, поменяем гипепараметры

best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train) # обучим модель на тренировочной выборке
        predictions_valid = model.predict(features_valid) # получим предсказания модели на валидационной выборке
        result = model.score(features_valid, target_valid) 
        if result > best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth
print("Accuracy наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", depth)   

Accuracy наилучшей модели на валидационной выборке: 0.8087091757387247 Количество деревьев: 40 Максимальная глубина: 10


In [9]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(5, 100, 10):
    for depth in range (1, 15):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train) 
        predictions_valid = model.predict(features_valid) 
        result = model.score(features_valid, target_valid) 
        if result > best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth
print("Accuracy наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", depth)  

Accuracy наилучшей модели на валидационной выборке: 0.807153965785381 Количество деревьев: 45 Максимальная глубина: 14


In [10]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 200, 10):
    for depth in range (1, 16):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train) 
        predictions_valid = model.predict(features_valid) 
        result = model.score(features_valid, target_valid) 
        if result > best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth
print("Accuracy наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", depth)  

Accuracy наилучшей модели на валидационной выборке: 0.8087091757387247 Количество деревьев: 40 Максимальная глубина: 15


Метрики качества оценивают качество работы и выражаются в числовой форме. Метрики качества тесно связаны с исходной задачей классификации. Accuracy не может быть меньше нуля (все ответы неправильные) и больше единицы (все ответы правильные).

Проведено исследование моделей: решающее дерево, случайный лес и логистическая регрессия. Самое высокое качество у случайного леса. Accuracy модели случайного леса показала лучший результат. Провели улучшение качества модели случайного леса с помощью изменений гипепараметров.

## Проверка модели на тестовой выборке

In [11]:
# проведем тестирование модели

model = RandomForestClassifier(n_estimators=40, max_depth=10, random_state=12345) #  инициализируем модель с наилучшим значением Accuracy
model.fit(features_train, target_train) # обучим модель на тренировочной выборке
predictions_valid = model.predict(features_valid) # получим предсказания модели на валидационной выборке

result = model.score(features_valid, target_valid) # посчитаем значение Accuracy на валидационной выборке
print("Accuracy наилучшей модели на валидационной выборке:", result)

Accuracy наилучшей модели на валидационной выборке: 0.7962674961119751


Получено значение Accuracy более 0.75. 

## Проверка модели на адекватность

In [12]:
model = DummyClassifier(strategy = "most_frequent", random_state = 0)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
print(accuracy_score(target_valid, predictions_valid))

0.7060653188180405
